# Chapter7
## Model Performance and Validation

In [1]:
import os, sys, time
import numpy as np
from math import floor, ceil
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.patches import Patch, Rectangle
from matplotlib.lines import Line2D
from tqdm import tqdm
import h5py
# import python library
sys.path.append(os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'py'))
from sample import Sample
from dataset import Dataset
from database import Database
from dataset_simulation import DatasetSimulation
from cfar_detector import CFARDetector
from nn_detector import NeuralNetworkDetector
from yolo_detector import YOLODetector
from cfar_detector_generator import CFARDetectorGenerator
from nn_detector_generator import NNDetectorGenerator
from yolo_detector_generator import YOLODetectorGenerator

dataset_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','datasets', 'awdEvents1')
database_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','databases', 'awdEvents1')
result_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','results', 'awdEvents1')
dataset_sim_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','datasets', 'simulations', 'simple' ,'whistler')
site = 'marion'
t_res, f_res = 0.006395061728395062, 0.15503875968992248

database = Database(dataset_loc, database_loc, site)

linewidth = 2
font_title = 32
font_legend = 20
font_label = 24
font_ticks = 18
font = [font_title, font_legend, font_label, font_ticks, linewidth] # [None]*5

### 7.4 Model Comparison

### Bounding Boxes

In [5]:
#CCSW
t, t_params = 'zscore',None
n = None
f_min, f_max = 1.5, 9.5
N,G,k,Ts,Tl,X_dB = 12,10,13,5,3,0.5
train = False
cfar_detector_gen = CFARDetectorGenerator(dataset_loc, database_loc, dataset_sim_loc, result_loc, site, t_res, f_res)
cfar_detector_gen.dataset_cross_correlation_gen(transforms=t, transforms_params=t_params, f_min=f_min, f_max=f_max, 
                                               An=0.35, D0=80, magnitude=1, N=N, G=G, k=k, Ts=Ts, Tl=Tl, X_dB=X_dB,
                                               train=train, n=n,force=False)
data = cfar_detector_gen.dataset_cross_correlation_load(transforms=t, transforms_params=t_params, f_min=f_min, f_max=f_max, 
                                               An=0.35, D0=80, magnitude=1, N=N, G=G, k=k, Ts=Ts, Tl=Tl, X_dB=X_dB,
                                               train=train, n=n)
confusion_matrix = data['confusion matrix']
performance = cfar_detector_gen.performance(confusion_matrix)
print(confusion_matrix)
print(performance)

100%|██████████| 725/725 [03:03<00:00,  3.94it/s]

[2331, 242, 247, 0]
{'recall': 0.904, 'precision': 0.906, 'f1 score': 0.905, 'g measure': 0.905, 'false alarm': 0.094, 'misdetection': 0.096}


In [6]:
data['bboxes']

{'2013-02-05UT16:20:55.29507602.marion.vr2': array([[60.049,  2.2  ,  3.2  ,  1.5  ,  9.5  ]]),
 '2013-07-04UT19:30:11.66691930.marion.vr2': array([[60.674,  2.2  ,  3.2  ,  1.5  ,  9.5  ],
        [60.727,  2.3  ,  3.3  ,  1.5  ,  9.5  ]]),
 '2013-07-29UT18:29:45.01731922.marion.vr2': array([[59.587,  1.1  ,  2.1  ,  1.5  ,  9.5  ],
        [58.802,  1.7  ,  2.7  ,  1.5  ,  9.5  ],
        [61.638,  2.3  ,  3.3  ,  1.5  ,  9.5  ],
        [61.624,  2.5  ,  3.5  ,  1.5  ,  9.5  ]]),
 '2013-07-02UT14:41:44.68451914.marion.vr2': array([[60.602,  1.1  ,  2.1  ,  1.5  ,  9.5  ],
        [60.434,  2.2  ,  3.2  ,  1.5  ,  9.5  ]]),
 '2013-06-09UT07:36:41.93694609.marion.vr2': array([[58.637,  1.3  ,  2.3  ,  1.5  ,  9.5  ],
        [60.211,  2.2  ,  3.2  ,  1.5  ,  9.5  ]]),
 '2013-08-04UT23:04:13.34371914.marion.vr2': array([[62.122,  2.2  ,  3.2  ,  1.5  ,  9.5  ]]),
 '2013-06-09UT20:06:11.22357109.marion.vr2': array([[60.748,  2.2  ,  3.2  ,  1.5  ,  9.5  ],
        [61.089,  2.3  ,  3.3 

In [4]:
# YOLO
n = None
train = False
model_name = 'real_transfer_learning'
t,t_params = 'detrend','linear' 
t_model = model_name+'_'+str(t)+'_'+str(t_params)
yolo_detector_gen = YOLODetectorGenerator(dataset_loc, database_loc, result_loc, site, t,t_params, t_model)
yolo_detector_gen.dataset_yolo_classification_gen(train=train, n=n, thresh=0.8, time_err=1, force=False)
data = yolo_detector_gen.dataset_yolo_classification_load(train=train, n=n, thresh=0.8,time_err=1)
confusion_matrix = data['confusion matrix']
print(confusion_matrix)
performance = yolo_detector_gen.performance(confusion_matrix)
print(performance)
print()

100%|██████████| 725/725 [00:00<00:00, 9172.07it/s]

[2214, 47, 364, 0]
{'recall': 0.859, 'precision': 0.979, 'f1 score': 0.915, 'g measure': 0.917, 'false alarm': 0.021, 'misdetection': 0.141}



In [5]:
# CCWK
transforms, transforms_params = ['slice','zscore','scale'], [[1.5,9.5],[None],[0,1]]
N,G,k,Ts,Tl,X_dB = 12,10,13,5,3,0.5
pfa = (1/(1+((10**(X_dB/10))/(2*N))))**(2*N)
kernel = CFARDetector(dataset_loc, site, files[0]).load_kernel('sim', [0.35,80,1])
results = []
for i in range(5):
    start = time.time()
    sample_duration = 0
    files = database.get_train()[150*i:150*i+50]
    for file in tqdm(files):
        sample = CFARDetector(dataset_loc, site, file)
        sample_duration += sample.get_time()[-1]
        sample.detection_bounding_boxes(transforms,transforms_params, kernel, 'fusion_cfar', [N,G,k,Ts,Tl,pfa], threshold=0, time_err=1, duration=True)
    end = time.time()
    process_duration = end-start
    results.append([sample_duration, process_duration])
results = np.array(results)
ratios = results[:,1]/results[:,0]
print(ratios.mean(), ratios.std())

100%|██████████| 50/50 [00:43<00:00,  1.15it/s]

0.17878062890283392 0.016535292336003843


In [7]:
#SDCNN
sample = Sample(dataset_loc, site, file)
transforms, transforms_params = 'zscore', None
name = 'cnn-3conv-2dense-4dropout-100epochs'
nn = NeuralNetworkDetector(dataset_loc, database_loc, site, transforms, transforms_params)
model = nn.load_model(name=name)
scaler = nn.load_scaler(name=name)
results = []
for i in range(5):
    start = time.time()
    sample_duration = 0
    files = database.get_train()[150*i:150*i+50]
    for file in tqdm(files):
        sample = CFARDetector(dataset_loc, site, file)
        sample_duration += sample.get_time()[-1]
        nn.detection_bounding_boxes(file,thresh=0.8, concav=True, concav_window=0.02,concav_error=-0.0, time_error=1)
    end = time.time()
    process_duration = end-start
    results.append([sample_duration, process_duration])
results = np.array(results)
ratios = results[:,1]/results[:,0]
print(ratios.mean(), ratios.std())

100%|██████████| 50/50 [01:32<00:00,  1.85s/it]

0.37629323515071206 0.006583782812161268


In [5]:
# YOLO
yolo = YOLODetector()
model_data_type = "real"
model_base = 'yolov3-tiny'
transforms, transforms_params = 'detrend', 'linear'
model_name = 'real_transfer_learning_detrend_linear'
yolo.load_model(model_base, model_data_type, model_name)
results = []
for i in range(5):
    start = time.time()
    sample_duration = 0
    files = database.get_train()[150*i:150*i+50]
    for file in tqdm(files):
        sample = CFARDetector(dataset_loc, site, file)
        sample_duration += sample.get_time()[-1]
        yolo.detection_bounding_boxes(dataset_loc, site, file, transforms, transforms_params,thresh=.8)
    end = time.time()
    process_duration = end-start
    results.append([sample_duration, process_duration])
results = np.array(results)
ratios = results[:,1]/results[:,0]
print(ratios.mean(), ratios.std())

100%|██████████| 50/50 [00:02<00:00, 21.11it/s]

0.009705474877542968 0.0010280131464050551
